In [478]:
#imports
import boto3
from botocore.exceptions import ClientError #for debugging
from dotenv import load_dotenv #for env variables
import os
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [479]:
#load the variables/password/IDs from the .env file
load_dotenv()
#get the access and secret key 
access_key_id = os.getenv('ACCESS_KEY_ID')
secret_key = os.getenv('SECRET_ACCESS_KEY')

#declare bucket name and data folder name where data will be downloaded to
bucket_name = "mindex-data-analytics-code-challenge"
data_folder = "/Users/mady/mindex_data_analytics_challenge/data/"

In [480]:
#create s3 client
s3 = boto3.client('s3', aws_access_key_id=access_key_id, aws_secret_access_key=secret_key, region_name='us-east-1')

#file names to download
files = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']

#download each file
for file in files:
    try:
        s3.download_file(bucket_name, file, data_folder + file)
        print(file + " downloaded successfully")
    except Exception as e:
        print(file + "error: ", e)

bengals.csv downloaded successfully
boyd_receiving.csv downloaded successfully
chase_receiving.csv downloaded successfully
higgins_receiving.csv downloaded successfully


In [481]:
'''
Use the pandas library to load each CSV into its own dataframe.
'''
dataframes = {} #dictionary to hold the dataframes for each player
player_files = ['/Users/mady/mindex_data_analytics_challenge/data/boyd_receiving.csv', '/Users/mady/mindex_data_analytics_challenge/data/chase_receiving.csv', '/Users/mady/mindex_data_analytics_challenge/data/higgins_receiving.csv']

#put each file into a pandas dataframe
for file in player_files:
    csv = file.split('/data/')[1]
    player_name = csv.split('_')[0].capitalize()
    dataframes[player_name] = pd.read_csv(file)
    #add player name as a column
    dataframes[player_name]['Player'] = player_name

#load bengals data into a panda frame
dataframes['Bangals'] = pd.read_csv('/Users/mady/mindex_data_analytics_challenge/data/bengals.csv')

In [482]:
'''
Join/Merge all of the dataframes together to display one global table that shows
the three different receiver’s yards and touchdown (TD) data as well as every
game result. Be sure to include Opponent, Location, and Result fields from the
bengals.csv file.

Replace the ‘1.0’ or ‘0.0’ values in the Result field to display ‘Win’ or ‘Loss’,
respectively.
'''

#concat the player data frames
df = pd.concat([dataframes['Boyd'], dataframes['Higgins'], dataframes['Chase']])

#join to the bengals data frame
df = df.merge(dataframes['Bangals'], on='Week', how= 'right')
df[['Yards', 'TD']] = df[['Yards', 'TD']].fillna(0)

#replace the 1.0 and 0.0 values in the Result field
df = df.replace({'Result': {1: 'Win', 0: 'Loss'}})
df.index = range(1, len(df) + 1)

#add indexs column so it's in CSV

#replace empty yard values with 0 (int)
df['Yards'] = df['Yards'].replace('', 0).fillna(0).astype(int)
df['TD'] = df['Yards'].replace('', 0).fillna(0).astype(int)

#save the global dataframe to a csv file 
df.to_csv('/Users/mady/mindex_data_analytics_challenge/data/global.csv', index=False)

In [483]:
#get variaables from the .env file
db_host = os.getenv('DB_HOST')
db_username = os.getenv('USERNAME')
db_password = os.getenv('PASSWORD')
db_name = os.getenv('DB_NAME')
db_table_name = os.getenv('TABLE_NAME')
db_address = os.getenv('ADDRESS')

#db connection details 
#note address is the IP address from terminal running nslookup with the host given
#because the host was not working to let me connect
db = {
    'dbname': db_name,
    'user': db_username,
    'password': db_password,
    'host': db_address,
    'port': '5432'
}
connection_string = 'postgresql+psycopg2://' + db_username + ':' + db_password + '@' + db_address + '/' + db_name
#create SQLalchemy engine
engine = create_engine(connection_string)

#for debugging, prints my table
def print_my_table():
    try:
        with engine.connect() as connection:
            query = f"SELECT * FROM {db_table_name};"
            mb = pd.read_sql(query, con=connection.connection)
            return mb
    except Exception as e:
        print("error connecting to PostgreSQL DB: ", e)
        

In [489]:
#helper method to execute a query 
def execute_query_helper(query):
    try:
        conn = psycopg2.connect(**db)
        cursor = conn.cursor()
        alter_query = query
        cursor.execute(alter_query)
        conn.commit()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        #close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#delete contents from table 
execute_query_helper("""DROP VIEW if exists results;""")
execute_query_helper("""DELETE FROM madison_banaszak;""")

#remove columns
execute_query_helper(""" ALTER TABLE madison_banaszak
        DROP COLUMN Week,
        DROP COLUMN Yards,
        DROP COLUMN TD,
        DROP COLUMN Player,
        DROP COLUMN Opponent,
        DROP COLUMN Location,
        DROP COLUMN Result
                     """)
#add columms back in
execute_query_helper("""  ALTER TABLE madison_banaszak
        ADD COLUMN Week TEXT,
        ADD COLUMN Yards INT,
        ADD COLUMN TD INT,
        ADD COLUMN Player TEXT,
        ADD COLUMN Opponent TEXT,
        ADD COLUMN Location TEXT,
        ADD COLUMN Result TEXT
                     """)
print_my_table()

,week,yards,td,player,opponent,location,result


In [485]:
'''
Write data to postgresql DB
Set up a connection to view the database using DBeaver and ensure all relevant
data is present.
'''

#delete contents from db before insert
execute_query_helper(""" DELETE FROM madison_banaszak; """)
conn = psycopg2.connect(**db)

#create cursor object
curr = conn.cursor()

#open the csv file
with open('/Users/mady/mindex_data_analytics_challenge/data/global.csv', 'r') as f:
    #skip the header row
    next(f)
    #copy the csv file into the table
    curr.copy_from(f, 'madison_banaszak', sep=',')
    conn.commit()
    curr.close()
    conn.close()

print("Print out postgres database table after insert:")
print_my_table()

Print out postgres database table after insert:


,week,yards,td,player,opponent,location,result
0,PRE1,0,0,,TB,Away,Win
1,PRE2,0,0,,WSH,Away,Loss
2,PRE3,0,0,,MIA,Home,Loss
3,REG1,32,32,Boyd,MIN,Home,Win
4,REG1,58,58,Higgins,MIN,Home,Win
...,...,...,...,...,...,...,...
58,POST3,103,103,Higgins,KC,Away,Win
59,POST3,54,54,Chase,KC,Away,Win
60,POST4,48,48,Boyd,LAR,Neutral,Loss
61,POST4,100,100,Higgins,LAR,Neutral,Loss


In [486]:
'''
Write a SQL Query to show the total yards each receiver had
throughout the season as well as the team's record displayed as: “# of Wins - #
of Losses”. Your query should generate the following view:
'''
def print_select_helper(query):
    try:
        conn = psycopg2.connect(**db)
        cursor = conn.cursor()
        alter_query = query
        cursor.execute(alter_query)
        s = pd.read_sql(alter_query, conn)
        return s            
    except Exception as e:
        print(f"Error: {e}")
    finally:
        #close connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()


execute_query_helper("""DROP VIEW if exists results;""")
execute_query_helper(""" CREATE VIEW results AS
WITH yards AS (
    SELECT
        SUM(CASE WHEN Player = 'Boyd' THEN Yards ELSE 0 END) AS "Boyd Yards",
        SUM(CASE WHEN Player = 'Higgins' THEN Yards ELSE 0 END) AS "Higgins Yards",
        SUM(CASE WHEN Player = 'Chase' THEN Yards ELSE 0 END) AS "Chase Yards"
    FROM 
        madison_banaszak
),
wins_losses AS (
    SELECT 
        CONCAT(
            SUM(CASE WHEN Result = 'Win' THEN 1 ELSE 0 END), 
            ' - ', 
            SUM(CASE WHEN Result = 'Loss' THEN 1 ELSE 0 END)
        ) AS "Wins/Losses"
    FROM (
        SELECT DISTINCT 
            Week, 
            Result 
        FROM 
            madison_banaszak
    ) AS distinct_results
)
SELECT * 
FROM 
    yards 
CROSS JOIN 
    wins_losses;
                     """)
print(
    "\nResults of the query:\n"
    "This shows the total yards each receiver had throughout the season,\n"
    "as well as the team's record displayed as: Wins - Losses.\n"
)

print_select_helper(""" SELECT * FROM results;""") 


Results of the query:
This shows the total yards each receiver had throughout the season,
as well as the team's record displayed as: Wins - Losses.



,Boyd Yards,Higgins Yards,Chase Yards,Wins/Losses
0,938,1400,1823,14 - 10
